#### Module import

In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import cv2
import matplotlib.pyplot as plt

#### Generator

In [4]:
# skip connection
class ResidualBlock(nn.Module):
    def __init__(self, in_channels): # 인자로 in_channels 받는다.
        super(ResidualBlock, self).__init__() # 상속받은 Module 함수 사용.
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1), # padding을 양끝단을 반사시켜서 사용
            nn.Conv2d(in_channels, in_channels, 3),
            nn.InstanceNorm2d(in_channels),# mini-batch의 이미지 개별의 분포를 사용
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, in_chnnels, 3),
            nn.InstanceNorm2d(in_channels)
        )
    
    def forward(self, x) :
        return x + self.block(x) # identity x와 net(x)를 더해준다.

In [5]:
class GeneratorResNet(nn.Module):
    def __init__(self, in_channels, num_residual_blocks=9):
        super(GeneratorResNet, self).__init__()
        
        # 3*256*256 -> 64*256*256
        out_channels=64
        self.conv = nn.Sequential(
            nn.RefloctionPad2d(in_channels), # 3-padding
            nn.Conv2d(in_channels, out_channels, 2*in_channels+1), # 7*7 conv, # channel 변동없음.
            nn.InstanceNorm(out_channels),
            nn.ReLU(inplace=True)) # 기존 데이터 대체
        
        channels = out_chabbels
        
        # encoder structure
        # downsampling : 64*256*256 -> 
        self.down = []
        for _ in range(2): # 2회 반복
            out_channels = channels * 2 # 1-step : 64*2 = 128
            self.down += [              # 2-step : 128*2 = 256 
                nn.Conv2d(channels, out_channels, 3, stride=2, padding=1),
                # stride=2 1/2로 down size(feature map)
                nn.InstanceNorm2d(out_channels),
                nn.ReLU(inplace=True)
            ]
            channels = out_channels
        self.down = nn.Sequential(*self.down) # down model을 sequential에 묶기
        
        self.trans = [ResidualBlock(channels) for _ in range(num_residual_blocks)]
        # 9개의 residual block
        self.trans = nn.Sequential(*self.down)
        
        # upsampling
        self.up = []
        for _ in range(2):
            out_channels = in_channels // 2 # 늘어난 채널 수를 1/2씩 줄인다.
        self.up += [
            nn.Upsample(scale_factor=2), # binear interpolation
            nn.Conv2d(channels, out_channels, 3, stride=1, padding=1), # channel 변동없음.
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True),
        ]
        self.up = nn.Sequential(*self.up)
        
        self.out = nn.Sequential(
            nn.ReflectionPad2d(in_channels), # 7 padding
            nn.Conv2d(channels, in_channels, 2*in_channels+1),# channel 변동없음.
            nn.Tanh()
        )
        
    def forward(self, x):
        x = self.conv(x)
        x = self.down(x)
        x = self.trans(x)
        x = self.up(x)
        x = self.out(x)
        return

#### Discriminator

In [6]:
class Discriminator(nn.Module):
    def __init__(self, in_channels):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
            *self.block
        )